In [1]:
val spark2 = spark
import spark2.implicits._

In [2]:
def creatStream(port: Int, duration: Int) {
    val lines = (spark.readStream
                      .format("socket")
                      .option("host", "localhost")
                      .option("port", port)
                      .load())
    
    val counts = (lines.as[String]
                       .map(line => line.replaceAll("[^a-zA-Z0-9\\s]", ""))
                       .flatMap(line => line.split("\\s+"))
                       .filter(_.nonEmpty)
                       .groupByKey(_.toLowerCase)
                       .count()
                       .orderBy($"count(1)" desc))
    
    val query = (counts.orderBy($"count(1)" desc)
                   .writeStream
                   .outputMode("complete")
                   .format("console")
                   .start
                   .awaitTermination(duration))
}

In [3]:
// nc -lk 12345 < war_and_peace.txt

creatStream(12345, 10000)

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+--------+
|value|count(1)|
+-----+--------+
|  the|    6940|
|  and|    4286|
|   to|    3282|
|   of|    2612|
|    a|    2296|
|   he|    1976|
|  his|    1782|
|   in|    1671|
|  was|    1406|
| that|    1373|
| with|    1229|
|  you|    1089|
|    i|    1077|
|   it|    1074|
|  her|    1022|
|   at|     981|
|  had|     944|
|  not|     893|
|  him|     855|
|   as|     847|
+-----+--------+
only showing top 20 rows

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+--------+
|value|count(1)|
+-----+--------+
|  the|   34570|
|  and|   22159|
|   to|   16716|
|   of|   14991|
|    a|   10521|
|   he|    9809|
|   in|    8801|
|  his|    7967|
| that|    7813|
|  was|    7329|
| with|    5695|
|  had|    5354|
|   it|    5179|
|  her|    4700|
|  not|    4658|
|  him|    4574|
|   at|    4538|
|    i|    4106|
|  but|    4013